### %s : 파이썬에서 문자열 포맷팅을 하기 위한 구문, SQL 쿼리에도 사용 가능  

### sql 쿼리에서는?

INSERT INTO AIRPORT.DAILY_PSG  
(ARR_OR_DEP)  
VALUES  
(%s)  <- ARR_OR_DEP에 들어갈 값을 나중에 채워넣을 자리  

### python 코드에서는?
cursor.execute("INSERT INTO AIRPORT.DAILY_PSG (ARR_OR_DEP) VALUES (%s)", ("A",))

In [1]:
import mysql.connector
from mysql.connector import Error
from config import db_config

def create_connection():
    try:
        connection = mysql.connector.connect(**db_config.DATABASE_CONFIG)
        if connection.is_connected():
            print('연결 성공')
            return connection
    
    except Error as e:
        print(f"에러 발생: {e}")
        return None

In [2]:
connection = create_connection()
cursor = connection.cursor()

연결 성공


In [4]:
insert_query = """
INSERT INTO AIRPORT.DAILY_PSG
(ARR_OR_DEP, AIRPORT_NAME, YYYYMMDD, HH_HOUR, PASSENGER_AMT, PASSENGER_TEAM_AMT, TYPE_OF_FLIGHT, CONGETST_YN)
VALUES
(%s, %s, %s, %s, %s, %s, %s, %s)
"""

In [7]:
data = (
    'D',
    'GMP',
    '20240000',
    '06',
    '0',
    '2656',
    'Y',
    'D'
)

In [8]:
cursor.execute(insert_query, data)

In [9]:
connection.commit()

In [10]:
print(f"{cursor.rowcount} row inserted.")

1 row inserted.


In [13]:
# %(변수명)s
insert_query_withvalues = """
INSERT INTO AIRPORT.DAILY_PSG
(ARR_OR_DEP, AIRPORT_NAME, YYYYMMDD, HH_HOUR, PASSENGER_AMT, PASSENGER_TEAM_AMT, TYPE_OF_FLIGHT, CONGETST_YN)
VALUES
(%(ARR_OR_DEP)s, %(AIRPORT_NAME)s, %(YYYYMMDD)s, %(HH_HOUR)s, %(PASSENGER_AMT)s, %(PASSENGER_TEAM_AMT)s, %(TYPE_OF_FLIGHT)s, %(CONGETST_YN)s)
"""

In [14]:
data_withvalues = {
    'ARR_OR_DEP' : 'D',
    'AIRPORT_NAME' : 'GMP',
    'YYYYMMDD' : '20240000',
    'HH_HOUR' : '07',
    'PASSENGER_AMT' : '0',
    'PASSENGER_TEAM_AMT' : '2656',
    'CONGETST_YN' : 'Y',
    'TYPE_OF_FLIGHT' : 'D'
}

In [15]:
cursor.execute(insert_query_withvalues, data_withvalues)

In [16]:
connection.commit()

In [17]:
print(f"{cursor.rowcount} row inserted.")

1 row inserted.


In [18]:
import requests
from bs4 import BeautifulSoup
from config import ENCODING_KEY, DECODING_KEY

In [20]:
url = 'http://openapi.airport.co.kr/service/rest/dailyExpectPassenger/dailyExpectPassenger'
params ={'serviceKey' : DECODING_KEY, 'schDate' : '20240831', 'schAirport' : 'GMP'}

response = requests.get(url, params=params)
xml_content = response.content

# BeautifulSoup을 사용해서 XML 파싱
soup = BeautifulSoup(xml_content, 'html')

# items 내의 모든 item 요소들을 찾기
items = soup.find_all('item')

datas = []

for item in items:
    item_data = {data.name : data.text for data in item.find_all()}
    datas.append(item_data)

/opt/homebrew/anaconda3/envs/crawling_apidb/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


In [21]:
datas

[{'aod': 'D',
  'arp': 'GMP',
  'congestyn': 'Y',
  'hh': '06',
  'pcg': '0',
  'pct': '2656',
  'sdt': '20240831',
  'tof': 'D'},
 {'aod': 'A',
  'arp': 'GMP',
  'congestyn': 'N',
  'hh': '07',
  'pcg': '0',
  'pct': '173',
  'sdt': '20240831',
  'tof': 'D'},
 {'aod': 'D',
  'arp': 'GMP',
  'congestyn': 'Y',
  'hh': '07',
  'pcg': '0',
  'pct': '2258',
  'sdt': '20240831',
  'tof': 'D'},
 {'aod': 'D',
  'arp': 'GMP',
  'congestyn': 'N',
  'hh': '07',
  'pcg': '0',
  'pct': '315',
  'sdt': '20240831',
  'tof': 'I'},
 {'aod': 'A',
  'arp': 'GMP',
  'congestyn': 'N',
  'hh': '08',
  'pcg': '0',
  'pct': '914',
  'sdt': '20240831',
  'tof': 'D'},
 {'aod': 'D',
  'arp': 'GMP',
  'congestyn': 'N',
  'hh': '08',
  'pcg': '0',
  'pct': '1434',
  'sdt': '20240831',
  'tof': 'D'},
 {'aod': 'D',
  'arp': 'GMP',
  'congestyn': 'N',
  'hh': '08',
  'pcg': '0',
  'pct': '601',
  'sdt': '20240831',
  'tof': 'I'},
 {'aod': 'A',
  'arp': 'GMP',
  'congestyn': 'Y',
  'hh': '09',
  'pcg': '0',
  'pct': 

In [26]:
def insert_data_fromlist(datas, connection) :
    cursor = connection.cursor()

    insert_query_withvalues = """
        INSERT INTO AIRPORT.DAILY_PSG
        (ARR_OR_DEP, AIRPORT_NAME, YYYYMMDD, HH_HOUR, PASSENGER_AMT, PASSENGER_TEAM_AMT, TYPE_OF_FLIGHT, CONGETST_YN)
        VALUES
        (%(ARR_OR_DEP)s, %(AIRPORT_NAME)s, %(YYYYMMDD)s, %(HH_HOUR)s, %(PASSENGER_AMT)s, %(PASSENGER_TEAM_AMT)s, %(TYPE_OF_FLIGHT)s, %(CONGETST_YN)s)
        """
    
    for item in datas : 
        data_withvalues = {
            'ARR_OR_DEP' : item['aod'],
            'AIRPORT_NAME' : item['arp'],
            'YYYYMMDD' : item['sdt'],
            'HH_HOUR' : item['hh'],
            'PASSENGER_AMT' : item['pct'],
            'PASSENGER_TEAM_AMT' : item['pcg'],
            'CONGETST_YN' : item['congestyn'],
            'TYPE_OF_FLIGHT' : item['tof']
        }
        cursor.execute(insert_query_withvalues, data_withvalues)
    
    connection.commit()
    connection.close()

In [27]:
connection = create_connection()
insert_data_fromlist(datas, connection)

연결 성공
